In [1]:
import os
%set_env CUDA_PATH=e:\cuda\bin
%set_env CUDA_PATH_V11_2=e:\cuda\bin
%set_env PATH=%PATH%;e:\cuda\bin;e:\cuda\extras;e:\cuda\include;e:\cuda\lib
#!set
os.add_dll_directory(r'e:\cuda\bin')
import cv2
import glob
from os.path import relpath
from os.path import basename
from os.path import dirname
from os.path import join
from os.path import splitext
import json

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

env: CUDA_PATH=e:\cuda\bin
env: CUDA_PATH_V11_2=e:\cuda\bin
env: PATH=%PATH%;e:\cuda\bin;e:\cuda\extras;e:\cuda\include;e:\cuda\lib


In [2]:
#data_dir = relpath("""E:\lambda\labs\ds-test-2\models\synthetic_data\synthetic_data_for_pipeline_transform\data""",__file__)
data_dir ="""E:\lambda\labs\ds-test-2\models\synthetic_data\synthetic_data_for_pipeline_transform\data\*\*"""

examples = glob.glob(data_dir)
print(examples[0],len(examples))

E:\lambda\labs\ds-test-2\models\synthetic_data\synthetic_data_for_pipeline_transform\data\10\007880cf-c97e-425d-9577-96318042598d.json 22170


In [3]:
records = {join(dirname(fn),basename(fn).split(".")[0]) for fn in examples}
X_train =[ cv2.cvtColor(cv2.imread(x),cv2.COLOR_RGB2BGR) for x in examples if "X_input" in x ]
#y_train =[ y for y in examples if "y_label" in y ]
y_train =[ json.loads( open(y,'rb').read() )["y_label_points"] for y in examples if ".json" in y ]

X_train = np.array(X_train)
y_train = np.array(y_train).reshape((X_train.shape[0],8))

In [4]:
len(X_train)

7390

In [5]:
# load real data

path_real = r'E:\lambda\labs\ds-test-2\data\transcribed_stories\*\*\phase0\*'
real_set=glob.glob(path_real)

X_real =np.array([ cv2.cvtColor(cv2.imread(x),cv2.COLOR_RGB2BGR) for x in real_set if "X_input" in x ])
y_real =[ json.loads(open(y,'rb').read() )["y_label_points"] for y in real_set if ".json" in y ]
print(y_real)
y_real = np.array(y_real).reshape((X_real.shape[0],8))
print(X_real.shape,y_real.shape)
#print(X_real,y_real)


[[[28.4377358490566, 20.545454545454547], [190.5509433962264, 18.363636363636363], [187.4716981132075, 206.0], [27.169811320754715, 199.27272727272725]], [[28.256603773584903, 24.727272727272727], [190.00754716981132, 23.09090909090909], [190.5509433962264, 249.27272727272725], [25.72075471698113, 245.09090909090907]], [[32.60377358490565, 34.54545454545455], [190.9132075471698, 33.81818181818181], [186.20377358490563, 250.1818181818182], [34.23396226415094, 246.0]]]
(3, 256, 192, 3) (3, 8)


In [6]:
class loss_history(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_loss = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))

def display_history(hist):
    # plot history
    try:
        loss_p = hist.history['loss']
        val_loss_p = hist.history['val_loss']
    except AttributeError as e:
        loss_p = hist.losses
        val_loss_p = hist.val_loss

    plt.figure(figsize=(15,5))
    plt.plot(loss_p, label='MAE (training data)')
    plt.title('MAE for points')
    plt.ylabel('MAE value')
    plt.xlabel('No. epoch')
    plt.legend(loc="upper left")
    plt.yscale('log',base=2)
    plt.show()

    plt.figure(figsize=(15,5))
    plt.plot(val_loss_p, label='MAE (validation data)')
    plt.title('MAE for points')
    plt.ylabel('MAE value')
    plt.xlabel('No. epoch')
    plt.legend(loc="upper left")
    plt.yscale('log',base=2)
    plt.show()

In [7]:
img_inputs = keras.Input(shape=(256, 256-64, 3))
x=img_inputs
#x= layers.Dropout(.2)(x)
#x = layers.Conv2D(32,3,2)(x)
#x = layers.BatchNormalization()(x)
#x = layers.Activation("relu")(x)
#x = layers.MaxPool2D((3,3))(x)

#x = layers.Conv2D(16,3,2)(x)
#x = layers.BatchNormalization()(x)
#x = layers.Activation("relu")(x)
#x = layers.MaxPool2D((3,3))(x)

x = layers.Flatten()(x)

x = layers.Dense(32, activation="relu",dtype="float64")(x)
x = layers.Dense(32, activation="relu",dtype="float64")(x)
x = layers.Dense(4, activation="relu",dtype="float64")(x)
x = layers.Dense(32, activation="relu",dtype="float64")(x)
x = layers.Dense(32, activation="relu",dtype="float64")(x)
outputs = layers.Dense(8,)(x)
model = keras.Model(inputs=img_inputs, outputs=outputs, name="FC_Model")
model.summary()
model.compile(loss="MAE",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001))

with tf.device("GPU"):
    history = model.fit(X_train,y_train,epochs=5000,validation_data=(X_real,y_real),shuffle=True,batch_size=100)

Model: "FC_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 192, 3)]     0         
                                                                 
 flatten (Flatten)           (None, 147456)            0         
                                                                 
 dense (Dense)               (None, 32)                4718624   
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 4)                 132       
                                                                 
 dense_3 (Dense)             (None, 32)                160       
                                                                 
 dense_4 (Dense)             (None, 32)                105

KeyboardInterrupt: 

In [ ]:
display_history(history)

In [ ]:
img_inputs = keras.Input(shape=(256, 256-64, 3))
x = img_inputs
x = layers.Dropout(.05)(x)
x = layers.Conv2D(32,3,2)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.MaxPool2D((3,3))(x)

x = layers.Conv2D(16,3,2)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.MaxPool2D((3,3))(x)

x = layers.Flatten()(x)

x = layers.Dense(8, activation="relu",dtype="float64")(x)
#x = layers.Dense(8, activation="relu",dtype="float64")(x)
#x = layers.Dense(8, activation="relu",dtype="float64")(x)
#x = layers.Dense(8, activation="relu",dtype="float64")(x)
#x = layers.Dense(8, activation="relu",dtype="float64")(x)
#x = layers.Dense(8, activation="relu",dtype="float64")(x)
#x = layers.Dense(8, activation="relu",dtype="float64")(x)
#x = layers.Dense(8, activation="relu",dtype="float64")(x)
#x = layers.Dense(8, activation="relu",dtype="float64")(x)
#x = layers.Dense(8, activation="relu",dtype="float64")(x)
outputs = layers.Dense(8,)(x)
model = keras.Model(inputs=img_inputs, outputs=outputs, name="FC_Model")
model.summary()
model.compile(loss="MAE",optimizer=tf.keras.optimizers.Adam(learning_rate=.0001))

with tf.device("GPU"):
    history =   model.fit(X_train,y_train,epochs=500,validation_data=(X_real,y_real),shuffle=True,batch_size=100)

In [ ]:
display_history(history)

In [59]:
img_inputs = keras.Input(shape=(256, 256-64, 3))
x = img_inputs
x = layers.Dropout(.05)(x)
#x = layers.Conv2D(32,3,2)(x)
#x = layers.BatchNormalization()(x)
#x = layers.Activation("relu")(x)
#x = layers.MaxPool2D((3,3))(x)

#x = layers.Conv2D(16,3,2)(x)
#x = layers.BatchNormalization()(x)
#x = layers.Activation("relu")(x)
#x = layers.MaxPool2D((3,3))(x)

x = layers.Flatten()(x)

x = layers.Dense(32, activation="relu",dtype="float64")(x)
x = layers.Dense(32, activation="relu",dtype="float64")(x)
x = layers.Dense(8, activation="relu",dtype="float64")(x)
x = layers.Dense(32, activation="relu",dtype="float64")(x)
x = layers.Dense(32, activation="relu",dtype="float64")(x)

outputs = layers.Dense(8,)(x)
model = keras.Model(inputs=img_inputs, outputs=outputs, name="FC_Model")
model.summary()
model.compile(loss="MAE",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001))

with tf.device("GPU"):
    history =None
    for _ in range(100):
        new_history = model.fit(X_train,y_train,epochs=5,validation_data=(X_real,y_real),shuffle=True,batch_size=100)
        if history is not None:
            for category in history.history.keys():
                print(category)
                history.history[category].append(new_history.history[category][0])
                print(history.history)
        else:
            history=new_history

        display_history(history)

Model: "FC_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 256, 192, 3)]     0         
                                                                 
 dropout_17 (Dropout)        (None, 256, 192, 3)       0         
                                                                 
 flatten_25 (Flatten)        (None, 147456)            0         
                                                                 
 dense_168 (Dense)           (None, 32)                4718624   
                                                                 
 dense_169 (Dense)           (None, 32)                1056      
                                                                 
 dense_170 (Dense)           (None, 8)                 264       
                                                                 
 dense_171 (Dense)           (None, 32)                288

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [60]:
history.history



AttributeError: 'NoneType' object has no attribute 'history'